In [ ]:
cd ~/Ali/FBB/Propensity

In [ ]:
import os
n_cpu= os.cpu_count() print(n_cpu)


In [ ]:
import pymysql
from sqlalchemy import create engine
import pandas as pd
import numpy as np
import seaborn as sns
import feature_engine
import psycopg2
import warnings
import mysql.connector
import inspect
import matplotlib.pyplot as plt 
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 40) 
pd.set_option('display.width', 120) 
import sqlalchemy as sa
from sqlalchemy.engine 
import url as sa_url 
from datetime import datetime 
import timeit
start_time = timeit.default_timer()

In [ ]:
ai = {}
ai['user'] = 'user'
ai['password'] = 'pwd'
ai['name'] = 'AI'
ai['host'] = '10.10.10.10'
ai['port'] = 3333

prod = {}
prod['user'] = 'user'
prod['password'] = 'pwd'
prod['name'] = 'Prod'
prod['host'] = '10.10.10.10'
prod['port'] = 3333

arch = {}
arch['user'] = 'user'
arch['password'] = 'pwd'
arch['name'] = 'Archival'
arch['host'] = '10.10.10.10'
arch['port'] = 3333

In [ ]:
def create_gp_engine():
  username = 'user',
  password = 'pwd',
  drivername = 'postgresql+psycopg2',
  host = '1.2.3.4',
  port = '1111',
  database = 'gpafiniti'

  url = sa_url.URL(drivername = drivername, host=host, port=port, database=database, username=username, password=password)

  return sa.create_engine(url)

def create_engine_arch_server(database):

  conn = psycopg2.connect(
      host = '1111',
      database = database, 
      user = 'user',
      password = 'pwd',
      port = 1111)
  return conn

def run_query(cred,db,query):
  try:
    mydb=mysql.connector.connect(
        host = cred['host'],
        port = cred['port'],
        user = cred['user'],
        password = cred['password'],
        database = db)
    
    df = pd.read_sql(quey,mydb)
    mydb.close()
    return df
  except Exception as e:
    print(str(e))

def update_query(cred, db, query): 
  try:
    mydb = mysql.connector.connect(
      host=cred['host'],
      port=cred['port'],
      user=cred['user'],
      password=cred['password'],
      database=db)
    
    mycursor = mydb.cursor()
    mycursor.execute(query)
    mydb.commit()
    mydb.close()
    return print(mycursor.rowcount, "record(s) affected")
  except Exception as e:
    print(str(e))

In [ ]:
script_pars ={
    'explanation': 'Eliminated btn and some pivot variables, btns had high NA percentage',
    'chunksize': 50000,
    'skipdbread': 0, #If you already read DB succesfully and wrote into model_folder as csv batches, you can skip this part and write as csv part by making it 1 
    'skippropwrite': 0, #If you already wrote propensity outputs into model folder as csv file batches, you can skip this part too
                        #0: write everything, owerwrite can occur
                        #1: If your kernel died during run, and you are sure input set didnt change, you can set it 1 to prevent overwrite, it wont save if name already there 
                        #2: skip everything, do not write any result, go on with what is already there.
    'rmallpropcsv': 0, #if 1, it removes every result csv in directory. Be careful while using it
    'nb_batches': 20, #If you already wrote data as csv batches, and you are skipping read DB part, you need to define nb of csv batches here, how many csv files do you have in directory 
    'sme_limit': ';', #Use it if you want to limit nrows for sme query. Otherwise just '' 
    'database': 'afiniti_aida_fbb', #sme database
    'table_to_use': 'afiniti_aida_fbb.lookups joined', #name of the sme
    'run_type': 'prod', #adds a suffix to models folder, you can use 'prod' or 'test'
    'top_n_models': 3, #please use the number you used when you were training the model. I will flag used models in training pipe, but for now it should be manual
    'preferred_folder_name': '2304061913-prod' #If you have a folder name containing pickle files you want to make predictions write it. " is suggested, to get latest created one on run type

In [ ]:
model_dir = os.getcwd() + "/model_results/" + script_pars['preferred_folder_name'] + "/" 

columns_dict = {}
with open(model_dir+"/columns.txt", 'r') as f:
  columns_dict = f.read()

In [ ]:
import ast
dictt = ast.literal_eval(columns_dict) 
ordered cols= dictt['ordered_cols']
cat_cols = dictt['cat_cols']
num_cols = dictt['num_cols']
print(dictt['ordered_cols'])

In [ ]:
#we have a large amount of variables, so we need to handle them manually in preprocessing. 
#this csv file keeps variable names, their real dtypes and decide whether they should be discarded or not 
var_df = pd.read_csv('varlist_fbb_new.csv', na_values = '', delimiter = ', ') 
var_df['name'] = var_df ['name'].str.lower()

var_df.loc[:, 'discard'] = 1
var_df.loc[var_df['name'].isin(['dsl_no']), 'discard'] = 0

var_df.loc[var_df['name'].isin(dictt['ordered_cols']), 'discard'] = 0

var_drop = list(var_df['name'][var_df['discard']==1])

var_df.drop(var_df.loc[var_df['name'].isin (var_drop) ==True].index, inplace=True)
var_df.reset_index(drop=True, inplace=True)
print(var_df.head())

str_vars=", ".join(var_df['name'])
print(str_vars)

In [ ]:
#pre_df['dsl_no'] = pre_df['dsl_no'].apply(lambda x: + str(x) + "*")
#dsl_vars=", ".join(pre_df['dsl_no'])
#print (pre_df.head())

if script_pars['skipdbread'] != 1:
  #your query
  query = """select """ + str_vars + """ from """ + script_pars['table_to_use'] + """ """ + script_vars['sme_limit']+ """ ;""" 
  #print(query)
  conn = create_gp_engine()
  dfs= []
  chunksize=script_pars['chunksize']
  i=0
  for chunk in pd.read_sql_query(query, con con = conn, chunksize=chunksize):
    print(i)
    dfs.append(chunk)
    i=i+1
    pre_df= pd.concat(dfs)
else:
  pass

In [ ]:
import pickle
model_dir = os.getcwd() + "/model_results/" + script_pars['preferred_folder_name'] + "/"

voting_dict = {}

with open(model_dir+"/voting_results.txt", 'r') as f:
  voting_str = f.read()

with open(model_dir+"/voting_results.pkl", 'rb') as f: 
  voting_dict = pickle.load(f)

with open(model_dir+"/voting_df.pkl", 'rb') as f: 
  voting_df = pickle.load(f)

#model_success_df = pd.read_csv(model_dir + 'voting_models.csv', na_values = '', delimiter = ')
print(voting_df)

In [ ]:
with open(model_dir+"/bins.pkl", 'rb') as f: 
  bins_dict = pickle.load(f)
print(bins_dict)

In [ ]:
top_n_models = script_pars['top_n_models']

voter_models = voting_df['model'].iloc[:top_n_models].unique()
voter_models.sort()
print(voter_models)

In [ ]:
from glob import glob
preferred_folder_name = script_pars['preferred_folder_name']

if preferred_folder_name == '';
  #model_dirs = [x[0] for x in os.walk (os.getcwd())]
  model_dirs = glob(os.getcwd()+"/model_results/*/", recursive = True)
  model_dirs_filtered = [x for x in model_dirs if script_pars['run_type'] in x] 
  model_dirs_filtered.sort(reverse = True)
  model_dir = model_dirs_filtered[0]

  model_list = [s for s in os.listdir(model_dir) if any (x in s for x in voter_models)]

elif preferred_folder_name != '':
  model_dir = os.getcwd() + "/model_results/" + preferred_folder_name + "/"
  model_list = [s for s in os.listdir (model_dir) if any (x in s for x in voter_models)]

In [ ]:
models = []
for i in range(len(model_list)):
  with open(model_dir+"/"+model_list[i], 'rb') as f:
    model = pickle.load(f)
    models.append(model)

In [ ]:
if script_pars['skipdbread'] != 1:
  import math
  print(pre_df.head())
  print(len(pre_df))
  batch_nb = math.ceil(len(pre_df)/1000000) 
  print(batch_nb)

  for i in range(batch_nb):
    print(i)
    sub_df = pre_df.iloc[i*1000000: (i+1)*1000000]
    sub_df.to_csv(model_dir+"/batch_"+str(i)+".csv", index = False)
    print(len(sub_df))
else:
  batch_nb = script_pars['batch_nb']

In [ ]:
'skipdbread': 0, #If you already read DB succesfully and wrote into model_folder as csv batches, you can skip this part and write as csv part by making it 1 
'skippropwrite': 0, #If you already wrote propensity outputs into model_folder as csv file batches, you can skip this part too
                    #0: write everything, owerwrite can occur
                    #1: If your kernel died during run, and you are sure input set didnt change, you can set it 1 to prevent overwrite, it wont save if name already there 
                    #2: skip everything, do not write any result, go on with what is already there.
'rmallpropcsv': 0, #if 1, it removes every result csv in directory. Be careful while using it

#If you set to remove all result csvs in the beginnning id does that
if script_pars['rmallpropcsv'] == 1:
  filelist = [f for f in os.listdir (model_dir) if f.endswith(".csv")] 
  proplist = [f for f in filelist if f.startswith("propensity_")] 
  for f in proplist:
    os.remove(os.path.join(model_dir, f))

if (script_pars['skippropwrite'] == 0) or (script_pars['rmallpropcsv'] == 1): 
  batcher = range(batch_nb)
  filelist = [f for f in os.listdir(model_dir) if f.endswith(".csv")] 
  proplist = [f for f in filelist if f.startswith("propensity_")]

elif script_pars['skippropwrite'] == 1:
  ###try to get numbers from propensity names as a list and ignore them

In [ ]:
for batch in range(batch_nb):
  print(batch)
  sub_df = pd.read_csv(model_dir+"/batch_"+str(batch)+".csv") 
  print(len(sub_df))
  print(sub_df.head())
  sub_df.columns = sub_df.columns.str.lower()
  var_drop = [col.lower() for col in var_drop]
  sub_df.drop(columns = [col for col in var_drop if col in sub_df.columns), axis= 1, inplace = True)

  sub_df = sub_df.replace('NA', np.nan)
  sub_df = sub_df.replace('NULL', np.nan)

  #var_df['type'] = np. where(war_df['name'].isin([xxxx]), 'category', var_df['type']) 
  var_df['type'] = np.where(var_df['type'].isin(['int']), 'Int64', var_df['type']) 
  print(sub_df.isna().sum())

  pre_df_cols = sub_df.columns
  varlist = list(var_df ['name'].str.lower())
  print(len(pre_df_cols))
  
  var_df.loc[var_df['name'].isin(['feata', 'featb']), 'type'] = 'category'
  sub_df['featc'] = sub_df['featc'].replace('4 GB', '4096') 
  sub_df['featc'] = sub_df['featc'].replace('6 GB', '6144') 
  sub_df['featc'] = sub_df['featc'].replace('Limitsiz', '204800')

  #changing python dtypes and if they are different than our list, change dtype. Float thing was not necessary, but after so many errors inside pipe, i needed to do it 
  for c in range(len(pre_df_cols)):
    print(c)
    if var_df.loc[c, 'type'] == 'datetime64':
      #main_df[main_df_cols[i]] = pd.to_datetime (main_df[main_df_cols[i]], format = var_df.loc[i, 'timetype'], errors = 'coerce')
      pass
    if var_df.loc[c, 'type'] == 'Int64':
      print(pre_df_cols[c])
      print(var_df.loc[c, 'type'])
      sub_df[pre_df_cols[c]] = sub_df[pre_df_cols[c]].astype('float')
      sub_df[pre_df_cols[c]] = sub_df[pre_df_cols[c]].astype(var_df.loc[c, 'type'])
    else:
      print(pre_df_cols[c])
      print(var_df.loc[c, 'type'])
      sub_df[pre_df_cols[c]] = sub_df[pre_df_cols[c]].astype(var_df.loc[c, 'type'])

  if [m for m in ['feata', 'featb'] if m in sub_df.columns]: 
    print("It is in colummns")
    ### !!!!!!! This is an important concept with 2 outcomes to be used in future. This cell is only for test purpose. 
    ### Replace is not workint with INPLACE parameter anymore, you need to specify it explicitly
    ### And if the DTYPE is not OBJECT, replacing NP. NAN value with a fixed string value is not working. You need to change DTYPE pre_df_test = sub_df.copy()
    pre_df_test[['feata', 'featb']] = pre_df_test[['feata', 'featb']].replace(np.nan, 'NA') 
    #print(pre_df_test[['feata', 'featb']].head (10))

    pre_df_test[['feata', 'featb']] = pre_df_test[['feata', 'featb']].astype('object') 
    pre_df_test[['feata','featb']] = pre_df_test[['feata', 'featb']].replace(np.nan, 'NA')
    #print(pre_df_test[['feata", 'featb']].head(10))

    ### Here is the 2nd variatoin, using fillna. But it needs to see it as a new category
    ### And also add_category is only using series, one column each time. So, i used a lambda funtion to apply it properly
    #These features are filled with NA because features are populated only if there is a cancellation application. So, missing values have meaning here

    sub_df[['feata', 'featb']] = sub_df [['feata', 'featb']].apply(lambda x: x.cat.add_categories('NA').fillna('NA')) 
    #print(value[['feata','featb']].head())

  ### Decided to fill all categorical NaN values with NA category.
  #print(value.dtypes.astype (str).value_counts())
  date_cols = list(sub_df.select_dtypes(include = ['datetime64']).columns)
  
  #num_cols = list(pre_df.select_dtypes (include = ['float', 'int', 'int64', 'Float64', 'Int64', 'float64']).columns)
  num_cols = dictt['num_cols']
  #cat_cols = list(pre_df.select_dtypes (exclude = ['float', 'int', 'int64', 'Float64', 'Int64', 'float64', 'datetime64']).columns) 
  cat_cols = dictt['cat_cols']
  
  sub_df.drop(columns = date_cols, axis= 1, inplace=True)

  pre_df1 = sub_df.iloc[:500000,]
  pre_df2 = sub_df.iloc[500000:.]

  #!! category dtype do not let you replace or fill np.nan's with new category value, that is why i switched to str type 
  #divided df into 2 as it was having memory error
  for col in cat_cols:
    pre_dfl[col] = pre_dfl[col].astype('object')
    pre_dfl[col] = pre_dfl[col].replace (np.nan, 'NA')

  for col in cat_cols:
    pre_df2[col] = pre_df2 [col].astype('object')
    pre_df2[col] = pre_df2[col].replace(np.nan, 'NA')

  sub_df = pre_df1.append(pre_df2, ignore_index = True)

  #print(value.dtypes.astype(str).value_counts())

  if [i for i in ['feat_a'] if i in sub_df.columns]:
    sub_df['feat_a'] = sub_df['feat_a'].astype('str').replace(r'^\+', '', regex=True)
    sub_df['feat_a'] = sub_df['feat_a'].astype('object')

  main_df = sub_df.reindex(columns= dictt['ordered_cols'])

  # read_csv broke the behavior of df, changedd numerical categorical values, added .0 to their end
  # It is for correcting them 
  for col in cat_cols:
    main_df[col] = main_df[col].astype('str').replace(r'\.0$', '', regex=True)

  main_dsl_df = sub_df.loc[:, 'dsl_no']

  inference = pd.DataFrame()
  for md in range(len(models)):
    inference['proba' + str(md)] = models[md].predict_proba(main_df)[:,1]
    inference['predict' + str(md)] = models[md].predict(main_df)
  
  inference['avg_proba'] = inference[[col for col in inference.columns if 'proba' in col]].mean(axis=1)
  inference['voted_is_sale'] = np.where(inference[[col for col in inference.columns if 'predict' in col]].sum(axis=1)/len(models) <0.5, 0,1) 
  #inference inference.join(df_y_final['is_sale'])
  #print (inference)

  dsl_df = pd.DataFrame(main_dsl_df, columns = ['dsl_no'])
  inference['dsl_no'] = dsl_df.loc[:, 'dsl_no']
  groups_list = [10,20,40,80]

  for ii in range(len(groups_list)):
    inference["PropensityGroup_bin "+str(groups_list[ii])] = pd.cut(inference["avg_proba"], bins=np.array(bins_dict[groups_list[ii]]), labels=False, include_lowest=True) 
  
  publish_df = pd.DataFrame()
  publish_df = inference.loc[:, ['dsl_no', 'avg_proba', 'PropensityGroup_bin10', 'PropensityGroup_bin20', 'PropensityGroup_bin40', 'PropensityGroup_bin80']] 
  print(publish_df.head(5))
  print(len(publish_df))

  publish_df.to_csv(model_dir+"/propensity_"+str(batch)+".csv", index = False)


In [ ]:
#### If val_test.csv added into training pipeline, copy paste above cell. Delete the for loop. Fetch the csv file and run this cell to analyze it. Check consistenct with training pipeline

sub_df = pd.read_csv(model_dir+"/val_test.csv")
###
###
###
###

In [ ]:
publish_df_all = pd.DataFrame()

for batch in range(batch_nb):
  print(batch)
  sub_df = pd.read_csv(model_dir+"/propensity_"+str(batch)+".csv")
  publish_df_all = sub_df.append(publish_df_all, ignore_index = True) 
  print(len(publish_df_all))

In [ ]:
#import sqlalchemy as sa
#from sqlalchemy import create_engine
#from sqlalchemy.engine import url as sa_url

#def create_mysql_engine(host, database, username, password, port='3307'):
  #url = sa_url.URL(drivername="mysql+pymysql', host=host, port=port, database=database,
  #username=username, password=password)
  #print(url)
  #return sa.create_engine(url)

#engine=create_mysql_engine (ai['host'], 'smexplorerdata_fbb', ai['user'], ai['password'], ai['port'])
#publish_df_all.to_sql(if_exists = 'replace', name= 'fbb_retention_propensity_groups_crm_test', con = engine, schema= 'smexplorerdata_fbb', index = False)

In [ ]:
#engine=create_mysql_engine (ai['host'], 'smexplorerdata_fbb', ai['user'], ai['password'], ai['port'])
#with engine.connect() as con:
#con.execute('CREATE INDEX fbb_retention_propensity_groups_crm_test_dsl_no_IDX USING BTREE ON smexplorerdata_fbb.fbb_retention_propensity_groups_crm_test (dsl_no (100));')

In [ ]:
import sqlalchemy as sa
from sqlalchemy import create engine
from sqlalchemy.engine import url as sa_url
print(publish_df_all)
conn = create_gp_engine()
publish_df_all.to_sql("propensity_feed_v2", con-conn, chunksize=10000, method= "multi", schema= "afiniti_aida_fbb", if_exists= "replace", index= False)

In [ ]:
engine=create_gp_engine()
with engine.connect() as con:
  con.execute('CREATE INDEX propensity_feed_v2_dsl_no_idx ON afiniti_aida_fbb.propensity_feed_v2 (dsl_no);')